# Workflow 1, Module 3a

Given a disease, first find chemical agents that make the disease worse. Then look for genes that decrease the amount of or response to the toxicant, and try to increase the amount or action of those genes.

In [12]:
input_disease = 'MONDO:0005148'  #Type 2 diabetes

## Functions

In [13]:
import requests
import pandas as pd

robokop_server = 'robokop.renci.org'

def quick(question):
    url=f'http://{robokop_server}:80/api/simple/quick/'
    response = requests.post(url,json=question)
    print( f"Return Status: {response.status_code}" )
    if response.status_code == 200:
        return response.json()
    return response

In [14]:
def make_N_step_question(types,curies,props,forwards):
    question = {
                'machine_question': {
                    'nodes': [],
                    'edges': []
                }
            }
    for i,t in enumerate(types):
        newnode = {'id': f'n{i}', 'type': t}
        if curies[i] is not None:
            newnode['curie'] = curies[i]
        question['machine_question']['nodes'].append(newnode)
        if i > 0:
            if forwards[i-1]:
                edge = {'id': f'e{i}', 'source_id': f'n{i-1}', 'target_id': f'n{i}'}
            else:
                edge = {'id': f'e{i}', 'source_id': f'n{i}', 'target_id': f'n{i-1}'}
            if props[i-1] is not None:
                edge['type'] = props[i-1]
            question['machine_question']['edges'].append( edge )
    return question

In [15]:
def merge_answers(answerdict):
    """Take a dictionary of answer lists and return a single sorted answer list."""
    final_answer = {'misc_info': {'natural_question': None, 'num_total_paths': 0}, 'answers': []}
    for k,v in answerdict.items():
        if 'filename' in v: 
            final_answer['filename'] = v['filename']
        if 'timestamp' in v:
            final_answer['timestamp'] = v['timestamp']
        if 'misc_info' in v:
            final_answer['misc_info']['num_total_paths'] += v['misc_info']['num_total_paths']
        if 'answers' in v:
            final_answer['answers'].extend(v['answers'])
    final_answer['answers'].sort(key = lambda a: -a['score'])

    return final_answer

In [16]:
import pandas as pd
def answers2frame(graph_answers,properties_a,properties_b):
    answers = []
    for graph_answer in graph_answers['answers']:
        try:
            #The return nodes don't necessarily come out in order. We can recognize the disease and the gene
            # by type, but we have to get the original toxicant by finding which node has the correct id.
            if len(graph_answer['nodes']) < 4:
                continue
            for edge in graph_answer['edges']:
                if edge['type']=='contributes_to':
                    toxicant_id = edge['source_id']
                elif edge['type'] in properties_a:
                    gene_tox = edge
                elif edge['type'] in properties_b:
                    chem_gene = edge
            for node in graph_answer['nodes']:
                if node['type'] == 'gene':
                    gene = node
                elif node['type'] == 'chemical_substance':
                    if node['id'] == toxicant_id:
                        toxicant=node
                    else:
                        drug=node
            ans = { 'score': graph_answer['score'],
                    'toxicant': toxicant['name'] if 'name' in toxicant else toxicant['id'],
                    'gene': gene['name'] if 'name' in gene else gene['id'],
                    'chemical': drug['name'] if 'name' in drug else drug['id'],
                    'gene->toxicant': gene_tox['type'],
                    'chem->gene': chem_gene['type']}
        except IndexError:
            #this happens if chemical and toxicant are the same thing
            continue
        answers.append(ans)
    df = pd.DataFrame(answers)
    ordered_columns = ['score','chemical','chem->gene','gene','gene->toxicant','toxicant']
    df = df[ordered_columns]
    return df

## Set up the property lists

There are several properties at each step that are acceptable for our purposes. 

In [17]:
properties_a=['increases_degradation_of',
              'decreases_abundance_of',
              'decreases_response_to',
              'increases_uptake_of']
properties_b=['increases_activity_of',
              'increases_expression_of',
              'decreases_degradation_of',
              'increases_stability_of',
              'increases_synthesis_of',
              'increases_secretion_of'
                ]
types = ['disease',
        'chemical_substance',
        'gene',
        'chemical_substance']
curies = [input_disease,None,None,None]
forwards = [False,False,False]

In [18]:
single_q = make_N_step_question(types,curies,['contributes_to',properties_a,properties_b],forwards)
single_answer = quick(single_q)

Return Status: 200


In [19]:
df = answers2frame(single_answer,properties_a,properties_b)

In [20]:
from IPython.display import display
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

,score,chemical,chem->gene,gene,gene->toxicant,toxicant
0,1.927600,metformin,increases_expression_of,SLC2A4,increases_uptake_of,glucose
1,1.713251,cholesterol,increases_activity_of,MYLK,increases_uptake_of,glucose
2,1.543229,glucose,increases_secretion_of,IL6,increases_uptake_of,zinc atom
3,1.493152,glucose,increases_secretion_of,IL1B,increases_uptake_of,zinc atom
4,1.487819,glucose,increases_secretion_of,IL6,increases_uptake_of,manganese atom
5,1.463625,rosiglitazone,increases_expression_of,SLC2A4,increases_uptake_of,glucose
6,1.458299,glucose,increases_expression_of,PTGS2,decreases_response_to,zinc atom
7,1.426568,rosiglitazone,increases_expression_of,UCP1,decreases_response_to,glucose
8,1.418777,glucose,increases_secretion_of,TNF,increases_uptake_of,iron atom
9,1.409878,glucose,increases_activity_of,EGF,increases_uptake_of,zinc atom


The first hit is for metformin, a known diabetes drug, and it's because it increases the uptake of glucose (the toxicant in this case).  The gene is interesting, because it is SLC2A4, rather than the expected INS (insulin).  This is probably due to the source of the data.

Both glucose and cholesterol also show up in the chemical column, which is meant to be the potential treatment!   This seems counterintuitive, but what we are seeing is that the query is too simple.  The question posits a linear relationship: there is a toxicant, the action of a gene gets rid of the toxicant, and the treatment makes the gene do this more. 

But actual biology is not so simple!  Specifically, feedback loops substantially complicate the picture.  If a gene is involved in cleaning up a toxicant, then the presence of that toxicant usually stimulates the gene.  So it's no suprise to see glucose on the treatment side, as it does tend to stimulate the metabolic processes that are disregulated in diabetes.

We don't have the ability to model things so directly, but we can at least filter out rows where the chemical has a known causal effect on diabetes.  We'd also like to flag answers that are already known diabetes treatments.

In [21]:
#Get chemicals that contribute to the disease
cq=make_N_step_question(['disease','chemical_substance'],[input_disease,None],['contributes_to'],[False])
aq=quick(cq)

Return Status: 200


In [22]:
contributors = []
for answer in aq['answers']:
    nodes = answer['nodes']
    for node in nodes:
        if node['type'] == 'chemical_substance':
            contributors.append(node)
df_contributors = pd.DataFrame(contributors)
df_contributors = df_contributors[['name']]
df_contributors['contributors']=True

In [23]:
#Get treatments for the disease
tq=make_N_step_question(['disease','chemical_substance'],[input_disease,None],['treats'],[False])
ta=quick(tq)
treatments = []
for answer in ta['answers']:
    nodes = answer['nodes']
    for node in nodes:
        if node['type'] == 'chemical_substance':
            treatments.append(node)
df_treatments = pd.DataFrame(treatments)
df_treatments = df_treatments[['name']]
df_treatments['treatments']=True

Return Status: 200


In [24]:
#Merge in the contributors and treatments.
dfx = pd.merge(df,df_contributors,left_on='chemical',right_on='name',how='left')
dfx = pd.merge(dfx,df_treatments,left_on='chemical',right_on='name',how='left')
dfx = dfx.sort_values(by='score',ascending=False)
dfx.fillna(False,inplace=True)
del dfx['name_x']
del dfx['name_y']

It's messy and counterintiutive, but we need to be aware that sometimes the same chemical can be listed as both a contributor and a treatment:

In [27]:
dfx[(dfx.contributors==True) & (dfx.treatments==True)].head()

,score,chemical,chem->gene,gene,gene->toxicant,toxicant,contributors,treatments
29,1.215675,glyburide,increases_secretion_of,INS,increases_uptake_of,glucose,True,True
50,0.817963,zinc atom,increases_secretion_of,TNF,increases_uptake_of,iron atom,True,True
56,0.774992,zinc atom,increases_secretion_of,TNF,increases_uptake_of,cadmium atom,True,True
60,0.769019,chromium atom,increases_secretion_of,IL6,increases_uptake_of,zinc atom,True,True
63,0.765397,zinc atom,increases_activity_of,MTF1,decreases_response_to,cadmium atom,True,True


In [29]:
dfx_no_contrib = dfx[ (dfx.contributors == False) | (dfx.treatments == True)]
dfx_no_contrib

,score,chemical,chem->gene,gene,gene->toxicant,toxicant,contributors,treatments
0,1.927600,metformin,increases_expression_of,SLC2A4,increases_uptake_of,glucose,False,True
1,1.713251,cholesterol,increases_activity_of,MYLK,increases_uptake_of,glucose,False,False
5,1.463625,rosiglitazone,increases_expression_of,SLC2A4,increases_uptake_of,glucose,False,True
7,1.426568,rosiglitazone,increases_expression_of,UCP1,decreases_response_to,glucose,False,True
13,1.365866,metformin,increases_activity_of,CAT,decreases_response_to,streptozocin,False,True
14,1.359609,metformin,increases_activity_of,CAT,decreases_response_to,glutathione,False,True
16,1.314547,cholesterol,increases_expression_of,APP,increases_uptake_of,CALCIUM,False,False
19,1.287761,rosiglitazone,increases_activity_of,SOD2,decreases_response_to,glucose,False,True
21,1.274303,lipid,increases_activity_of,AHR,decreases_response_to,N-nitrosodiethylamine,False,False
23,1.266932,metformin,increases_activity_of,CAT,decreases_response_to,L-ascorbic acid,False,True


Just to make this a little easier to read, let's just keep the first row whenever we find a new chemical.  So for instance, we'll keep the first metformin above, but not any others. This potentially hides some extra evidence for a chemical, so use with caution, but it makes it easier to get an overall view of what chemicals are found.  It's ok to get the first row because the frame is already sorted by score.

In [30]:
dfx_no_contrib.groupby('chemical').nth(0).reset_index().sort_values(by='score',ascending=False)

,chemical,chem->gene,contributors,gene,gene->toxicant,score,toxicant,treatments
24,metformin,increases_expression_of,False,SLC2A4,increases_uptake_of,1.927600,glucose,True
8,cholesterol,increases_activity_of,False,MYLK,increases_uptake_of,1.713251,glucose,False
33,rosiglitazone,increases_expression_of,False,SLC2A4,increases_uptake_of,1.463625,glucose,True
21,lipid,increases_activity_of,False,AHR,decreases_response_to,1.274303,N-nitrosodiethylamine,False
19,glyburide,increases_secretion_of,True,INS,increases_uptake_of,1.215675,glucose,True
12,cortisol,increases_expression_of,False,INS,increases_uptake_of,1.078811,glucose,False
29,pioglitazone,increases_activity_of,False,PPARG,decreases_response_to,1.059292,N-nitrosodiethylamine,True
32,reactive oxygen species,increases_activity_of,False,TNF,increases_uptake_of,1.048185,iron atom,False
40,tolbutamide,increases_expression_of,False,SLC2A1,increases_uptake_of,0.986397,glucose,True
17,ethanol,increases_activity_of,False,SOD2,decreases_response_to,0.963019,glucose,False


Our attempt to remove known contributors was mostly successful.  However, it was imperfect because these annotations of contributor and treatment are imperfect.  In particular, while things like HDL and LDL are listed as contributors, the more general terms cholesterol and lipid are not.  Other answers that should probably be understood as finding chemicals that make diabetes worse are cortisol (a stress-hormone that increases glucose) and reactive oxygen species, including hydrogen peroxide.   Let's remove all of these as well.

In [31]:
moretox = ['cholesterol','lipid','cortisol','reactive oxygen species','hydrogen peroxide']
newdfx = dfx_no_contrib[ ~dfx_no_contrib.chemical.isin(moretox)]
#And choose first row of each chemical
newdfx.groupby('chemical').nth(0).reset_index().sort_values(by='score',ascending=False)

,chemical,chem->gene,contributors,gene,gene->toxicant,score,toxicant,treatments
20,metformin,increases_expression_of,False,SLC2A4,increases_uptake_of,1.927600,glucose,True
28,rosiglitazone,increases_expression_of,False,SLC2A4,increases_uptake_of,1.463625,glucose,True
17,glyburide,increases_secretion_of,True,INS,increases_uptake_of,1.215675,glucose,True
25,pioglitazone,increases_activity_of,False,PPARG,decreases_response_to,1.059292,N-nitrosodiethylamine,True
35,tolbutamide,increases_expression_of,False,SLC2A1,increases_uptake_of,0.986397,glucose,True
15,ethanol,increases_activity_of,False,SOD2,decreases_response_to,0.963019,glucose,False
37,troglitazone,increases_expression_of,False,SLC2A1,increases_uptake_of,0.935249,glucose,True
39,zinc atom,increases_secretion_of,True,TNF,increases_uptake_of,0.817963,iron atom,True
23,phlorizin,increases_stability_of,False,SLC5A1,increases_uptake_of,0.770882,glucose,False
8,chromium atom,increases_secretion_of,True,IL6,increases_uptake_of,0.769019,zinc atom,True


### Notes on the rows that are not already known treatments 

Ethanol: Ethanol can reduce glucose levels, but it's a complicated picture, because it can also increase glucose.  Long-term ethanol can also lead to reduced insulin sensititivy.

Phlorizin: Wikipedia says: Phlorizin was studied as a potential pharmaceutical treatment for type 2 diabetes, but has since been superseded by more selective and more promising synthetic analogs, such as empagliflozin, canagliflozin and dapagliflozin.[8][9] Phlorizin is not an effective drug because when orally consumed, it is nearly entirely converted into phloretin by hydrolytic enzymes in the small intestine.[10][11]

Cobalt: https://www.ncbi.nlm.nih.gov/pubmed/16802070 says "We conclude that chronic cobalt treatment decreases plasma glucose levels in STZ-diabetic rats and improves tolerance to glucose."  Also https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3138145/

ATP: ATP is involved in cell-energy cycles along with glucose

Carbon: ?

Dexamethasone: glucocorticosteroid used to treat asthma, COPD, and many other things. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5503862/ says: "Glucocorticoids are widely used as potent anti-inflammatory and immunosuppressive drugs to treat a wide range of diseases. However, they are also associated with a number of side effects, including new-onset hyperglycemia in patients without a history of diabetes mellitus (DM) or severely uncontrolled hyperglycemia in patients with known DM."

Zinc Oxide: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3958835/: "Antidiabetic Activity of Zinc Oxide and Silver Nanoparticles on Streptozotocin-Induced Diabetic Rats"

Lipopolysaccharide: https://www.ncbi.nlm.nih.gov/pubmed/23835694 "Elevated levels of lipopolysaccharide (LPS) predict incident diabetes"

Dopamine: https://www.sciencedirect.com/science/article/pii/S1751991815001540 "A review of dopamine agonist therapy in type 2 diabetes and effects on cardio-metabolic parameters"  Also https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5945822/

Phosphate: ?

Biguanide: is actually a class of drugs that includes metformin.

Testosterone: Testosterone has been investigated as a diabetic therapy for men, but results seem mixed

doxorubicin: a chemotherapy drug. https://onlinelibrary.wiley.com/doi/full/10.1002/jcsm.12104: "Doxorubicin caused severe hyperglycaemia and insulin resistance, mediated by inhibition in AMPk signalling in skeletal muscle"

17beta-estradiol: female sex hormone.  There's a relation to diabetes & it's been looked at as a therapy, but again results seem mixed, and possibly dependent on menopause

acetylcholine: neurotransmitter involved in glucose homeostasis.  https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4113066/

Melatonin: Hormone involved in sleep (and used to treat insomnia).  There's some association with T2D, but it's complicated: https://link.springer.com/article/10.1007/s00125-017-4255-x

terbutaline: beta2 agonist for asthma.  Known to cause hyperglycemia and exacerbate diabetes.  

progesterone: Also a female sex hormone.  A lot of work in this area, but unclear what the relationship is.

phenol: polyphenols have in-vitro antioxidative effects, but also low bioavailability and may or may not have in vivo antioxidant effects 

azide: https://www.sciencedirect.com/science/article/pii/S2211383512000949 lists azide as a mitochondrial inhibitor that may increase insulin sensitivity

quercetin: a flavonoid. https://www.sciencedirect.com/science/article/pii/S0753332218349187 "In vitro and in vivo evidence that quercetin protects against diabetes and its complications: A systematic review of the literature"

valproic acid: anticonvulsant. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5483554/ "Does Valproic Acid Have Potential in the Treatment of Diabetes Mellitus?"  https://www.ncbi.nlm.nih.gov/pubmed/27079868 "Valproic Acid Improves Glucose Homeostasis by Increasing Beta-Cell Proliferation, Function, and Reducing its Apoptosis through HDAC Inhibition in Juvenile Diabetic Rat."

rotenone: an isoflavone used as a pesticide, but: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5783883/ "Rotenone ameliorated hyperglycaemia and insulin resistance in db/db mice. It induced glucose consumption and glycolysis and reduced hepatic glucose output. Rotenone also activated AMPK. Furthermore, it remained effective with AMPK inactivation."

arachidonic acid: a polyunsaturated fat.  Wikipedia:Furthermore, higher concentrations of ARA in muscle tissue may be correlated with improved insulin sensitivity.[ https://www.ncbi.nlm.nih.gov/pubmed/8418404 ].

simvastatin: statins tend to increase diabetic risk, so this probably goes in the contributors column. Note that the toxicant for this row is another statin.

silicon dioxide: ?

cyclosporin: an immunosuppressant.  There is a post-transplant diabetes that happens when tacrolimus is used as an immunosuppressant, and cyclosporin seems to be better?  https://onlinelibrary.wiley.com/doi/abs/10.1111/ajt.14665

fenofibrate: cholesterol medication.

### Rare Diseases

The above was all carried out for type 2 diabetes, a common disease.  Here we try to run the same thing but for rare forms of diabetes (as found in WF1Mod1_Ontology.ipynb). We're going to try a lot, because the data is sparse for these diseases, and we will see that most of them do not return any results.

In [33]:
rare_diseases = ['MONDO:0014785',
'MONDO:0011986',
'MONDO:0012819',
'MONDO:0012513',
'MONDO:0014458',
'MONDO:0012970',
'MONDO:0013478',
'MONDO:0013242',
'MONDO:0009575',
'MONDO:0016391',
'MONDO:0010802',
'MONDO:0016422',
'MONDO:0012971',
'MONDO:0011955',
'MONDO:0015308',
'MONDO:0014991',
'MONDO:0009517',
'MONDO:0013647',
'MONDO:0011363',
'MONDO:0014523',
'MONDO:0011386',
'MONDO:0017230',
'MONDO:0008763',
'MONDO:0012966',
'MONDO:0012436',
'MONDO:0012348',
'MONDO:0012520',
'MONDO:0007455',
'MONDO:0008812',
'MONDO:0020569',
'MONDO:0018105',
'MONDO:0012818',
'MONDO:0007669',
'MONDO:0014686',
'MONDO:0007452',
'MONDO:0013225',
'MONDO:0007453',
'MONDO:0012480',
'MONDO:0009874',
'MONDO:0013240',
'MONDO:0010020',
'MONDO:0012962',
'MONDO:0012923',
'MONDO:0011502',
'MONDO:0010800',
'MONDO:0014589',
'MONDO:0012963',
'MONDO:0008696',
'MONDO:0011027',
'MONDO:0011273',
'MONDO:0013673',
'MONDO:0018581',
'MONDO:0018911',
'MONDO:0011073',
'MONDO:0010773',
'MONDO:0011667',
'MONDO:0014488',
'MONDO:0011668',
'MONDO:0012192',
'MONDO:0020525',
'MONDO:0019207',
'MONDO:0010785',
'MONDO:0009192',
'MONDO:0018320',
'MONDO:0015967',
'MONDO:0014674',
'MONDO:0000208',
'MONDO:0018625',
'MONDO:0008185',
'MONDO:0009419',
'MONDO:0010894',
'MONDO:0011643',
'MONDO:0012071',
'MONDO:0009101',
'MONDO:0000065',
'MONDO:0018575',
'MONDO:0012969',
'MONDO:0014497',
'MONDO:0018629',
'MONDO:0018883',
'MONDO:0012522',
'MONDO:0010026',
'MONDO:0011072',
'MONDO:0008491']

In [38]:
outputs = {}
for rare_id in rare_diseases:
    rarecuries = [rare_id,None,None,None]
    single_disease_question = make_N_step_question(types,rarecuries,['contributes_to',properties_a,properties_b],forwards)
    rare_answer = quick(single_disease_question)
    if rare_answer == 'No results found':
        npaths = 0
    else:
        npaths = rare_answer['misc_info']['num_total_paths']
    print(rare_id,npaths)
    if npaths > 0:
        outputs[rare_id] = rare_answer

Return Status: 200
MONDO:0014785 0
Return Status: 200
MONDO:0011986 0
Return Status: 200
MONDO:0012819 250
Return Status: 200
MONDO:0012513 0
Return Status: 200
MONDO:0014458 0
Return Status: 200
MONDO:0012970 0
Return Status: 200
MONDO:0013478 0
Return Status: 200
MONDO:0013242 0
Return Status: 200
MONDO:0009575 0
Return Status: 200
MONDO:0016391 0
Return Status: 200
MONDO:0010802 0
Return Status: 200
MONDO:0016422 0
Return Status: 200
MONDO:0012971 0
Return Status: 200
MONDO:0011955 0
Return Status: 200
MONDO:0015308 0
Return Status: 200
MONDO:0014991 0
Return Status: 200
MONDO:0009517 0
Return Status: 200
MONDO:0013647 0
Return Status: 200
MONDO:0011363 0
Return Status: 200
MONDO:0014523 0
Return Status: 200
MONDO:0011386 0
Return Status: 200
MONDO:0017230 0
Return Status: 200
MONDO:0008763 0
Return Status: 200
MONDO:0012966 0
Return Status: 200
MONDO:0012436 0
Return Status: 200
MONDO:0012348 0
Return Status: 200
MONDO:0012520 0
Return Status: 200
MONDO:0007455 0
Return Status: 200

In [39]:
outputs.keys()

dict_keys(['MONDO:0012819', 'MONDO:0008491'])

We only received any responses for `MONDO:0012819` (diabetic ketoacidosis) and `MONDO:0008491` (stiff-person syndrome).  Let's merge the responses.

In [41]:
rare_answers = merge_answers(outputs)
df = answers2frame(rare_answers,properties_a,properties_b)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

,score,chemical,chem->gene,gene,gene->toxicant,toxicant
0,0.563518,dopamine,increases_activity_of,DRD3,decreases_response_to,clozapine
1,0.431542,acetone,increases_activity_of,CYP2E1,decreases_response_to,chlorpromazine
2,0.418220,phenobarbital,increases_activity_of,CYP1A2,decreases_response_to,clozapine
3,0.412806,ethanol,increases_activity_of,CYP2E1,decreases_response_to,chlorpromazine
4,0.389815,paracetamol,increases_activity_of,CYP2E1,decreases_response_to,chlorpromazine
5,0.386653,benzene,increases_activity_of,CYP2E1,decreases_response_to,chlorpromazine
6,0.385268,caffeine,increases_activity_of,CYP1A2,decreases_response_to,clozapine
7,0.384670,paracetamol,increases_expression_of,CYP1A2,decreases_response_to,clozapine
8,0.383350,chlorpyrifos,increases_expression_of,CYP1A2,decreases_response_to,clozapine
9,0.383068,phenobarbital,increases_activity_of,CYP2E1,decreases_response_to,chlorpromazine


In [42]:
#Cull out rows with repeat chemicals
df.groupby('chemical').nth(0).reset_index().sort_values(by='score',ascending=False)

,chemical,chem->gene,gene,gene->toxicant,score,toxicant
86,dopamine,increases_activity_of,DRD3,decreases_response_to,0.563518,clozapine
38,acetone,increases_activity_of,CYP2E1,decreases_response_to,0.431542,chlorpromazine
132,phenobarbital,increases_activity_of,CYP1A2,decreases_response_to,0.418220,clozapine
90,ethanol,increases_activity_of,CYP2E1,decreases_response_to,0.412806,chlorpromazine
126,paracetamol,increases_activity_of,CYP2E1,decreases_response_to,0.389815,chlorpromazine
54,benzene,increases_activity_of,CYP2E1,decreases_response_to,0.386653,chlorpromazine
63,caffeine,increases_activity_of,CYP1A2,decreases_response_to,0.385268,clozapine
68,chlorpyrifos,increases_expression_of,CYP1A2,decreases_response_to,0.383350,clozapine
160,toluene,increases_activity_of,CYP2E1,decreases_response_to,0.382953,chlorpromazine
166,valproic acid,increases_expression_of,SCN1A,decreases_response_to,0.380509,carbamazepine


Overall, this looks like a somewhat less interesting list.  The toxicants are not endogenous chemicals, but are drugs that are probalby contra-indicated for these particular conditions.  So we might end up hooking into reasonable biology, but it's going to be somewhat indirectly.  But, we can make some notes:

dopamine: we noted above that several dopminergic compounds appeared in the list, and that dopamine is involved in glycemic control.

acetone: this is probably getting highly ranked because it will be a commonly occuring term with ketoacidosis (which causes the breath to smell of acetone).

phenobarbital: https://www.ncbi.nlm.nih.gov/pubmed/26348778  Phenobarbital reduces blood glucose and glucogenesis.

paracetamol: acetemenaphen / tylenol

benzene: carcinogenic.. https://academic.oup.com/toxsci/advance-article-abstract/doi/10.1093/toxsci/kfy252/5140220 Benzene exposure leads to insulin resistance in mice

caffeine: caffeine seems to affect glucose levels, probably badly in diabetics.

chlorpyrifos: an insecticide, exposure to which might be associated with diabetes

toluene: methylbenzene, used as a solvent

haloperidol: antipsychotic, tends to increase glucose level